<a href="https://colab.research.google.com/github/samsenko/quora-duplicates/blob/master/Quora_Duplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import spacy
import numpy as np
import tensorflow as tf
import keras
import os
import pandas as pd
if 'COLAB_TPU_ADDR' in os.environ:
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
else:
  TPU_WORKER = None

In [0]:
from google.colab import files
uploaded = files.upload()

Saving questions.csv to questions.csv


In [0]:
data = pd.read_csv('questions.csv', encoding = "ISO-8859-1").values

In [0]:
inputs = (data.T[:-1]).T
inputs1 = (inputs.T[3]).T
inputs2 = (inputs.T[4]).T
targets = data.T[-1]

In [0]:
#Perform embeddings

#uncomment next line if not already downloaded
!python -m spacy download en_core_web_md
embedding = spacy.load('en_core_web_md')
embedding_length = 300
max_question_length = 30

def embed_question(inputs, max_datapoints):
  global embedding_length, max_question_length
  embedded_inputs = []
  i = 0
  errors = 0
  for question in inputs:
    if i % 1000 == 0:
      print("i = ", i)
    if i == max_datapoints:
      break
    i += 1
    doc = embedding(question)
    padded = np.zeros((max_question_length, embedding_length))
    new_question = np.array([])
    for word in doc:
      new_question = np.vstack((new_question, word.vector)) if new_question.size else word.vector
    try:
      padded[:new_question.shape[0]] = new_question
    except:
      errors += 1
      padded[:new_question.shape[0]] = new_question[:max_question_length]
    embedded_inputs.append(padded)
  embedded_inputs = np.array(embedded_inputs)
  print("errors: ", errors)
  return embedded_inputs

emb1 = embed_question(inputs1, 50000)
emb2 = embed_question(inputs2, 50000)

    100% |████████████████████████████████| 120.9MB 79.3MB/s 
  Running setup.py install for en-core-web-md ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')

i =  0
i =  1000
i =  2000
i =  3000
i =  4000
i =  5000
i =  6000
i =  7000
i =  8000
i =  9000
i =  10000
i =  11000
i =  12000
i =  13000
i =  14000
i =  15000
i =  16000
i =  17000
i =  18000
i =  19000
i =  20000
i =  21000
i =  22000
i =  23000
i =  24000
i =  25000
i =  26000
i =  27000
i =  28000
i =  29000
i =  30000
i =  31000
i =  32000
i =  33000
i =  34000
i =  35000
i =  36000
i =  37000
i =  38000
i =  39000
i =  40000
i =  41000
i =  42000
i =  43000
i =  44000
i =  45000
i =  46000
i =  47000
i =  48000
i =  49000
i =  50000
errors:  992
i =  0
i =  1000
i =  2000
i =  3000
i =  4000
i =  5000
i =  6000
i =  7000
i =  8000
i =  9000
i = 

In [0]:
print(emb1.shape, emb2.shape, targets[:50000].shape)

(50000, 30, 300) (50000, 30, 300) (50000,)


In [0]:
np.savez_compressed('quora_embedded', a=emb1, b=emb2, c = targets[:50000])

In [0]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()


Exception ignored in: <bound method NpzFile.__del__ of <numpy.lib.npyio.NpzFile object at 0x7fdce7475b38>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/numpy/lib/npyio.py", line 210, in __del__
    self.close()
  File "/usr/local/lib/python3.6/dist-packages/numpy/lib/npyio.py", line 201, in close
    if self.zip is not None:
AttributeError: 'NpzFile' object has no attribute 'zip'



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

    media = MediaFileUpload(path, 
                    mimetype='application/octet-stream',
                    resumable=True)

    created = drive_service.files().create(body=file_metadata,
                                   media_body=media,
                                   fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created

save_file_to_drive("quora_embedded", "./quora-duplicates/quora_embedded.npz")

File ID: 1VHycjtP6NcpmPFyXJxMd-XykMG0dTpzj


{'id': '1VHycjtP6NcpmPFyXJxMd-XykMG0dTpzj'}

In [0]:
!git init 

Reinitialized existing Git repository in /content/.git/
